# Финальная сборка и заливка табличек с данными о потенциальных местах размещения постаматов

Потенциальные места размещения:
- многоквартирный дом
- киоск
- МФЦ
- библиотека
- дом культуры или клуб
- cпортивный объект

Данные, которые будут лежать в табличках:
- Административный округ
- Район
- Тип объекта размещения
- Координата точки размещения
- Адрес точки размещения

Исходные данные скачиваем в формате json из:
- Нестационарные торговые объекты по реализации печатной продукции: https://data.mos.ru/opendata/2781
- Нестационарные торговые объекты: https://data.mos.ru/opendata/619
- Многофункциональные центры предоставления государственных и муниципальных услуг https://data.mos.ru/opendata/-mnogofunktsionalnye-tsentry-predostavleniya-gosudarstvennyh-uslug
- Библиотеки города: https://data.mos.ru/opendata/7702155262-biblioteki
- Дома культуры и клубы: https://data.mos.ru/opendata/7702155262-doma-kultury-i-kluby
- Спортивные объекты города Москвы: https://data.mos.ru/opendata/7708308010-sportivnye-obekty-goroda-moskvy
- данные о жилых домах - из уже залитой в БД <code>apartment_houses_all_data</code>

Заливаются таблички:
- <code>kiosks_all_data</code>
- <code>mfc_all_data</code>
- <code>libs_all_data</code>
- <code>clubs_all_data</code>
- <code>sports_all_data</code>
- <code>all_objects_data</code>

Перед прогоном тетрадки, если надо обновить <code>apartment_houses_all_data</code>, нужно запустить подготовку данных:
- <code>step1_prepare_dmr_houses_data.ipynb</code>
- <code>step2_prepare_gis_houses_data.ipynb</code>
- <code>step3_apartment_houses_all_data.ipynb</code>

In [1]:
import os
from typing import List, Dict
from warnings import warn
import json
import pandas as pd
import numpy as np

from postamats.utils import prepare_data
from postamats.utils.connections import DB
from postamats.utils.prepare_data import OBJECT_ID_COL

pd.set_option('display.max_columns', None)

In [2]:
CONFIG_PATH = '/Users/affernus/PROJECTS_DATA/hacks/postomat_optimisation/db_config.json'
with open(CONFIG_PATH, mode='r') as db_file:
    db_config = json.load(db_file)

databaase = DB(db_config)

In [3]:
KIOSKS_FOLDER = '/Users/affernus/PROJECTS_DATA/hacks/postomat_optimisation/kiosks'
# Нестационарные торговые объекты
KIOSKS_FILE_1 = 'data-104108-2022-10-23.json'
# Нестационарные торговые объекты по реализации печатной продукции
KIOSKS_FILE_2 = 'data-101756-2022-10-25.json'
# Многофункциональные центры предоставления государственных и муниципальных услуг 
MFC_FOLDER = '/Users/affernus/PROJECTS_DATA/hacks/postomat_optimisation/mfc'
MFC_FILE = 'data-2728-2022-10-26.json'
# Библиотеки
LIBS_FOLDER = '/Users/affernus/PROJECTS_DATA/hacks/postomat_optimisation/libs'
LIBS_FILE = 'data-7361-2022-10-19.json'
# дома культуры и клубы
CLUBS_FOLDER = '/Users/affernus/PROJECTS_DATA/hacks/postomat_optimisation/clubs'
CLUBS_FILE = 'data-7392-2022-10-27.json'
# спортивные объекты
SPORTS_FOLDER = '/Users/affernus/PROJECTS_DATA/hacks/postomat_optimisation/sports'
SPORTS_FILE = 'data-8303-2022-08-08.json'

In [4]:
COLS_MAP = {
    'AdmArea': 'ADM_AREA',
    'District': 'DISTRICT',
    'ObjectType': 'OBJECT_TYPE',
    'LAT': 'LAT',
    'LON': 'LON',
    'Address': 'ADDRESS',
    OBJECT_ID_COL: OBJECT_ID_COL,
    'Name': 'NAME',
    'ShortName': 'NAME',
    'CommonName': 'NAME'
}
MANDATORY_COLS = [OBJECT_ID_COL, 'ADM_AREA', 'DISTRICT', 'OBJECT_TYPE', 'LAT', 'LON', 'ADDRESS']

In [5]:
def create_lat_lon(data: pd.DataFrame, geodata_col: str) -> pd.DataFrame:
    data = data.copy()
    data[geodata_col] = data[geodata_col].apply(prepare_data.create_geolist)
    centroid = data[geodata_col].apply(prepare_data.calc_polygon_centroid)
    data['LAT'] = centroid.apply(lambda x: x[1] if isinstance(x, list) else np.nan)
    data['LON'] = centroid.apply(lambda x: x[0] if isinstance(x, list) else np.nan)
    return data

def extract_data_from_addr(addr_list: List[Dict[str, str]], addr_key: str) -> str:
    if not isinstance(addr_list, list):
        return np.nan
    if not addr_list:
        return np.nan
    if addr_key not in addr_list[0]:
        return np.nan
    return addr_list[0][addr_key]

def explode_address(data):
    data = data.copy()
    data['AdmArea'] = data['ObjectAddress'].apply(lambda x: extract_data_from_addr(x, 'AdmArea'))
    data['District'] = data['ObjectAddress'].apply(lambda x: extract_data_from_addr(x, 'District'))
    data['Address'] = data['ObjectAddress'].apply(lambda x: extract_data_from_addr(x, 'Address'))
    return data.drop(columns='ObjectAddress')

def extract_working_hours(hours_list: List[Dict[str, str]]) -> str:
    if not isinstance(hours_list, list):
        return np.nan
    if not hours_list:
        return np.nan
    return '; '.join([' '.join(el.values()) for el in hours_list])


def unify_colnames(data):
    cols = data.columns
    return [COLS_MAP[col] if col in COLS_MAP else col.upper() for col in cols]


In [6]:
kiosks1 = pd.read_json(os.path.join(KIOSKS_FOLDER, KIOSKS_FILE_1), encoding='cp1251').dropna(axis=1, how='all')
kiosks2 = pd.read_json(os.path.join(KIOSKS_FOLDER, KIOSKS_FILE_2), encoding='cp1251').dropna(axis=1, how='all')

mfc = pd.read_json(os.path.join(MFC_FOLDER, MFC_FILE), encoding='cp1251').dropna(axis=1, how='all')

libs = pd.read_json(os.path.join(LIBS_FOLDER, LIBS_FILE), encoding='cp1251').dropna(axis=1, how='all')

clubs = pd.read_json(os.path.join(CLUBS_FOLDER, CLUBS_FILE), encoding='cp1251').dropna(axis=1, how='all')

sports = pd.read_json(os.path.join(SPORTS_FOLDER, SPORTS_FILE), encoding='cp1251').dropna(axis=1, how='all')

In [7]:
kiosks = pd.concat([kiosks1, kiosks2], ignore_index=True)
print(kiosks.columns)
kiosks = create_lat_lon(kiosks, 'geoData')
kiosks_cols = ['Category', 'Name', 'AdmArea', 'District', 'Address',
              'FacilityArea', 'ObjectType', 'Specialization',
              'PeriodOfPlacement', 'StartDateTrading', 'EndDateTrading', 'LAT', 'LON']
kiosks['Category'] = kiosks['ObjectType'].copy()
kiosks['ObjectType'] = 'киоск'
kiosks['Specialization'] = kiosks['Specialization'].str.replace('[', '', regex=False)
kiosks['Specialization'] = kiosks['Specialization'].str.replace(']', '', regex=False)
kiosks = kiosks[kiosks_cols].copy()
kiosks[OBJECT_ID_COL] = prepare_data.get_row_hashes(kiosks)
kiosks.columns = unify_colnames(kiosks)
kiosks = kiosks.drop_duplicates(subset=[OBJECT_ID_COL])
kiosks.head(2)

Index(['Code', 'Name', 'AdmArea', 'District', 'Address', 'UNOM',
       'FacilityArea', 'ObjectType', 'Specialization', 'PeriodOfPlacement',
       'ContractType', 'ContractStatus', 'ContractNumber',
       'NameOfBusinessEntity', 'ContractBegin', 'ContractEnd', 'ContractBasis',
       'StartDateTrading', 'EndDateTrading', 'global_id', 'geoData',
       'geodata_center', 'ObjectPhoto'],
      dtype='object')


/Users/affernus/PROJECTS/hacks/postomat_optimisation/src/postamats/utils/prepare_data.py:169: UserWarning: coords is dot
  warn('coords is dot')


,CATEGORY,NAME,ADM_AREA,DISTRICT,ADDRESS,FACILITYAREA,OBJECT_TYPE,SPECIALIZATION,PERIODOFPLACEMENT,STARTDATETRADING,ENDDATETRADING,LAT,LON,OBJECT_ID
0,киоск,"Театральные билеты на Малый Чудов пер., вл. 8",Центральный административный округ,район Хамовники,"Российская Федерация, город Москва, внутригоро...",6.0,киоск,театральные билеты,с 1 января по 31 декабря,06.12.2021,06.12.2026,55.733460,37.592085,6b006db614c47fc5ea260e3450a250e9c8f3ad4606c802...
1,киоск,"Мороженое на Комсомольский просп., вл. 23-25",Центральный административный округ,район Хамовники,"Российская Федерация, город Москва, внутригоро...",6.0,киоск,мороженое,с 1 января по 31 декабря,10.01.2018,10.01.2023,55.726996,37.582864,8e649f104eecace30ea41bd80d70e69c97fac2d1356934...


In [8]:
databaase.load_to_bd(kiosks, 'kiosks_all_data')
databaase.get_table_from_bd('kiosks_all_data')

/Users/affernus/PROJECTS/hacks/postomat_optimisation/src/postamats/utils/connections.py:53: UserWarning: В названиях колонок есть uppercase: ['CATEGORY', 'NAME', 'ADM_AREA', 'DISTRICT', 'ADDRESS', 'FACILITYAREA', 'OBJECT_TYPE', 'SPECIALIZATION', 'PERIODOFPLACEMENT', 'STARTDATETRADING', 'ENDDATETRADING', 'LAT', 'LON', 'OBJECT_ID'] postgress имеет проблемы с uppercase, будет выполнен lower()
  warn(
/Users/affernus/PROJECTS/hacks/postomat_optimisation/src/postamats/utils/connections.py:59: UserWarning: Новые названия колонок: Index(['category', 'name', 'adm_area', 'district', 'address', 'facilityarea',
       'object_type', 'specialization', 'periodofplacement',
       'startdatetrading', 'enddatetrading', 'lat', 'lon', 'object_id'],
      dtype='object')
  warn(f'Новые названия колонок: {df.columns}')


Connection to PostgreSQL DB successful


/Users/affernus/PROJECTS/hacks/postomat_optimisation/src/postamats/utils/connections.py:75: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(f'select * from {table_name}', connection)


,category,name,adm_area,district,address,facilityarea,object_type,specialization,periodofplacement,startdatetrading,enddatetrading,lat,lon,object_id
0,киоск,"Театральные билеты на Малый Чудов пер., вл. 8",Центральный административный округ,район Хамовники,"Российская Федерация, город Москва, внутригоро...",6.0,киоск,театральные билеты,с 1 января по 31 декабря,06.12.2021,06.12.2026,55.733460,37.592085,6b006db614c47fc5ea260e3450a250e9c8f3ad4606c802...
1,киоск,"Мороженое на Комсомольский просп., вл. 23-25",Центральный административный округ,район Хамовники,"Российская Федерация, город Москва, внутригоро...",6.0,киоск,мороженое,с 1 января по 31 декабря,10.01.2018,10.01.2023,55.726996,37.582864,8e649f104eecace30ea41bd80d70e69c97fac2d1356934...
2,киоск,"Мороженое на Комсомольский просп., вл. 45",Центральный административный округ,район Хамовники,"Российская Федерация, город Москва, внутригоро...",6.0,киоск,мороженое,с 1 января по 31 декабря,10.10.2020,10.10.2025,55.720382,37.573841,a2dfe590fecc9ad92272bd5320b0f2d29a49e50006fc56...
3,киоск,"Мороженое на Ул. Остоженка, вл. 2",Центральный административный округ,район Хамовники,"Российская Федерация, город Москва, внутригоро...",6.0,киоск,мороженое,с 1 января по 31 декабря,10.10.2020,10.10.2025,55.743848,37.601243,10935af0f45663a3eb5faebfe9995bda8fdeb13839dc13...
4,киоск,"Мороженое ул. Новоалексеевская, вл. 5-7",Северо-Восточный административный округ,Алексеевский район,"город Москва, Новоалексеевская улица, дом 5, с...",6.0,киоск,мороженое,с 1 января по 31 декабря,27.04.2017,31.12.2023,55.807812,37.640956,16744787effb3a303fbb817d33f421366eda0b2e9d8173...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4324,киоск,"НТО Тушинская площадь, вл. 7, к. 3",Северо-Западный административный округ,район Покровское-Стрешнево,"город Москва, проезд Стратонавтов, дом 9, корп...",12.0,киоск,печать,с 1 января по 31 декабря,,,55.826418,37.437371,d042b6df5f4e9cb45abff46ac922e2f2646174ac1dd82f...
4325,киоск,"НТО Инженерная ул., вл. 1",Северо-Восточный административный округ,Алтуфьевский район,"Российская Федерация, город Москва, внутригоро...",9.0,киоск,печать,с 1 января по 31 декабря,,,55.879402,37.581948,4afcafc786b3b0aa158c0e81a8fd19c5a8327301d84485...
4326,киоск,НТО Липовый Парк (уч.1),Новомосковский административный округ,поселение Сосенское,"город Москва, поселение Сосенское, посёлок Ком...",6.0,киоск,печать,с 1 января по 31 декабря,,,55.567853,37.486937,98d7df65dd3d6b687d29346d0f76da8f776dc8d83bca61...
4327,киоск,НТО метро «Черкизовская»,Восточный административный округ,район Преображенское,"Российская Федерация, город Москва, внутригоро...",12.0,киоск,печать,с 1 января по 31 декабря,,,55.801777,37.744883,3a6a896cf96950ffa9af2e517918e573d088c7aa00d22c...


In [9]:
print(mfc.columns)
mfc = create_lat_lon(mfc, 'geoData')
mfc_cols = ['ShortName', 'AdmArea', 'District', 'Address', 'WorkingHours',
            'ExtraServices', 'CenterArea', 'WindowCount', 'LAT', 'LON']
mfc = mfc[mfc_cols].copy()
mfc['ObjectType'] = 'МФЦ'
mfc['WorkingHours'] = mfc['WorkingHours'].apply(extract_working_hours)
mfc[OBJECT_ID_COL] = prepare_data.get_row_hashes(mfc)
mfc.columns = unify_colnames(mfc)
mfc = mfc.drop_duplicates(subset=[OBJECT_ID_COL])
mfc.head(2)

Index(['CommonName', 'FullName', 'ShortName', 'INN', 'KPP', 'OGRN', 'AdmArea',
       'District', 'Address', 'ChiefName', 'ChiefPosition', 'PublicPhone',
       'WorkingHours', 'ClarificationOfWorkingHours', 'OpenDate',
       'ExtraServices', 'CenterArea', 'WindowCount', 'WebSite', 'UNOM',
       'Availability', 'global_id', 'IsOnBalance', 'geoData'],
      dtype='object')


/Users/affernus/PROJECTS/hacks/postomat_optimisation/src/postamats/utils/prepare_data.py:169: UserWarning: coords is dot
  warn('coords is dot')


,NAME,ADM_AREA,DISTRICT,ADDRESS,WORKINGHOURS,EXTRASERVICES,CENTERAREA,WINDOWCOUNT,LAT,LON,OBJECT_TYPE,OBJECT_ID
0,МФЦ района Красносельский,Центральный административный округ,Красносельский район,"Российская Федерация, город Москва, внутригоро...",понедельник 08:00-20:00; вторник 08:00-20:00; ...,"[фотоуслуги, копировальные услуги, оплата госу...",1024.6,54,55.785872,37.660717,МФЦ,08436cbd02c89a75906f767371e00008aa5148bf8f1d1b...
1,МФЦ района Сокол,Северный административный округ,район Сокол,"город Москва, Чапаевский переулок, дом 16",понедельник 08:00-20:00; вторник 08:00-20:00; ...,"[фотоуслуги, копировальные услуги, оплата госу...",1542.1,41,55.797147,37.518207,МФЦ,68a65daa7a14a431206ed69b1d8f682faa79a59f4a1c4e...


In [10]:
databaase.load_to_bd(mfc, 'mfc_all_data')
databaase.get_table_from_bd('mfc_all_data')

/Users/affernus/PROJECTS/hacks/postomat_optimisation/src/postamats/utils/connections.py:53: UserWarning: В названиях колонок есть uppercase: ['NAME', 'ADM_AREA', 'DISTRICT', 'ADDRESS', 'WORKINGHOURS', 'EXTRASERVICES', 'CENTERAREA', 'WINDOWCOUNT', 'LAT', 'LON', 'OBJECT_TYPE', 'OBJECT_ID'] postgress имеет проблемы с uppercase, будет выполнен lower()
  warn(
/Users/affernus/PROJECTS/hacks/postomat_optimisation/src/postamats/utils/connections.py:59: UserWarning: Новые названия колонок: Index(['name', 'adm_area', 'district', 'address', 'workinghours',
       'extraservices', 'centerarea', 'windowcount', 'lat', 'lon',
       'object_type', 'object_id'],
      dtype='object')
  warn(f'Новые названия колонок: {df.columns}')


Connection to PostgreSQL DB successful


/Users/affernus/PROJECTS/hacks/postomat_optimisation/src/postamats/utils/connections.py:75: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(f'select * from {table_name}', connection)


,name,adm_area,district,address,workinghours,extraservices,centerarea,windowcount,lat,lon,object_type,object_id
0,МФЦ района Красносельский,Центральный административный округ,Красносельский район,"Российская Федерация, город Москва, внутригоро...",понедельник 08:00-20:00; вторник 08:00-20:00; ...,"{фотоуслуги,""копировальные услуги"",""оплата гос...",1024.6,54,55.785872,37.660717,МФЦ,08436cbd02c89a75906f767371e00008aa5148bf8f1d1b...
1,МФЦ района Сокол,Северный административный округ,район Сокол,"город Москва, Чапаевский переулок, дом 16",понедельник 08:00-20:00; вторник 08:00-20:00; ...,"{фотоуслуги,""копировальные услуги"",""оплата гос...",1542.1,41,55.797147,37.518207,МФЦ,68a65daa7a14a431206ed69b1d8f682faa79a59f4a1c4e...
2,МФЦ района Ново-Переделкино,Западный административный округ,район Ново-Переделкино,"город Москва, Новопеределкинская улица, дом 12А",понедельник 08:00-20:00; вторник 08:00-20:00; ...,"{фотоуслуги,""копировальные услуги"",""оплата гос...",751.8,37,55.641319,37.366429,МФЦ,7f1036f5b19eff45e5d45ae4ce12ebb5bc98d4743df5ba...
3,МФЦ района Академический,Юго-Западный административный округ,Академический район,"Российская Федерация, город Москва, внутригоро...",понедельник 08:00-20:00; вторник 08:00-20:00; ...,"{фотоуслуги,""копировальные услуги"",""оплата гос...",1554.1,54,55.680727,37.580068,МФЦ,9c6cde90f227807edbc2319eaba9b93882214a7bda8e65...
4,МФЦ района Чертаново Южное,Южный административный округ,район Чертаново Южное,"город Москва, Россошанская улица, дом 4, корпус 2",понедельник 08:00-20:00; вторник 08:00-20:00; ...,"{фотоуслуги,""копировальные услуги"",""оплата гос...",1376.8,63,55.592468,37.609510,МФЦ,212d72249277fa3a3c391012cfa775c35450ad2102b530...
...,...,...,...,...,...,...,...,...,...,...,...,...
134,Флагманский офис ВАО,Восточный административный округ,район Гольяново,"Российская Федерация, город Москва, внутригоро...",понедельник 10:00-22:00; вторник 10:00-22:00; ...,"{фотоуслуги,""копировальные услуги"",""оплата гос...",2500.0,84,55.811034,37.800860,МФЦ,2040593daffdfe94efa6d0a949175dafaa9fc64b0a06bf...
135,МФЦ поселения Щербинка,Новомосковский административный округ,поселение Щербинка,"Российская Федерация, город Москва, город Щерб...",понедельник 10:00-22:00; вторник 10:00-22:00; ...,"{фотоуслуги,""копировальные услуги"",""оплата гос...",1500.0,67,55.504264,37.559168,МФЦ,3a7e8496810a1ddaf75390943fcacb942a378c08ee0cdd...
136,Флагманский офис ЮВАО,Юго-Восточный административный округ,Нижегородский район,"город Москва, Рязанский проспект, дом 2, корпус 3",понедельник 10:00-22:00; вторник 10:00-22:00; ...,"{фотоуслуги,""копировальные услуги"",""оплата гос...",2670.0,63,55.729313,37.734579,МФЦ,2fe6af50274ced77f55381b49a219605eb6d49979d8c1d...
137,Флагманский офис САО,Северный административный округ,Войковский район,"город Москва, Ленинградское шоссе, дом 16А, ст...",понедельник 10:00-22:00; вторник 10:00-22:00; ...,"{фотоуслуги,""копировальные услуги"",""оплата гос...",2500.0,61,55.824089,37.495536,МФЦ,f77050487a1cde130e5d31939394e15112993814c10f9e...


In [11]:
print(libs.columns)
libs = create_lat_lon(libs, 'geoData')
libs_cols = ['Category', 'CommonName', 'ObjectAddress', 'WorkingHours', 'NumOfSeats',
            'NumOfReaders', 'NumOfVisitors', 'LAT', 'LON']
libs = libs[libs_cols].copy()
libs['ObjectType'] = 'библиотека'
libs['WorkingHours'] = libs['WorkingHours'].apply(extract_working_hours)
libs = explode_address(libs)
libs[OBJECT_ID_COL] = prepare_data.get_row_hashes(libs)
libs.columns = unify_colnames(libs)
libs = libs.drop_duplicates(subset=[OBJECT_ID_COL])
libs.head(2)

Index(['Category', 'CommonName', 'FullName', 'ShortName', 'OrgInfo',
       'ObjectAddress', 'ChiefOrg', 'ChiefName', 'ChiefPosition',
       'PublicPhone', 'Fax', 'Email', 'WorkingHours',
       'ClarificationOfWorkingHours', 'WebSite', 'NumOfSeats', 'NumOfReaders',
       'NumOfVisitors', 'global_id', 'geoData'],
      dtype='object')


/Users/affernus/PROJECTS/hacks/postomat_optimisation/src/postamats/utils/prepare_data.py:172: UserWarning: coords is line
  warn('coords is line')
/Users/affernus/PROJECTS/hacks/postomat_optimisation/src/postamats/utils/prepare_data.py:169: UserWarning: coords is dot
  warn('coords is dot')


,CATEGORY,NAME,WORKINGHOURS,NUMOFSEATS,NUMOFREADERS,NUMOFVISITORS,LAT,LON,OBJECT_TYPE,ADM_AREA,DISTRICT,ADDRESS,OBJECT_ID
0,Библиотека,Библиотека №140,понедельник выходной; вторник 12:00-22:00; сре...,30.0,6103.0,45840.0,55.581234,37.684641,библиотека,Южный административный округ,район Бирюлёво Восточное,"Российская Федерация, город Москва, внутригоро...",6402778e01d99926d24a93a3ccff21168a932ce07a284f...
1,Библиотека,Библиотека №166 имени 1-го Мая,понедельник выходной; вторник 10:00-22:00; сре...,20.0,3002.0,25672.0,55.708619,37.586619,библиотека,Южный административный округ,Донской район,"Российская Федерация, город Москва, внутригоро...",405d8d977e6c63b8442a7fbb7deb2b32d289e02ff5dd9a...


In [12]:
databaase.load_to_bd(libs, 'libs_all_data')
databaase.get_table_from_bd('libs_all_data')

/Users/affernus/PROJECTS/hacks/postomat_optimisation/src/postamats/utils/connections.py:53: UserWarning: В названиях колонок есть uppercase: ['CATEGORY', 'NAME', 'WORKINGHOURS', 'NUMOFSEATS', 'NUMOFREADERS', 'NUMOFVISITORS', 'LAT', 'LON', 'OBJECT_TYPE', 'ADM_AREA', 'DISTRICT', 'ADDRESS', 'OBJECT_ID'] postgress имеет проблемы с uppercase, будет выполнен lower()
  warn(
/Users/affernus/PROJECTS/hacks/postomat_optimisation/src/postamats/utils/connections.py:59: UserWarning: Новые названия колонок: Index(['category', 'name', 'workinghours', 'numofseats', 'numofreaders',
       'numofvisitors', 'lat', 'lon', 'object_type', 'adm_area', 'district',
       'address', 'object_id'],
      dtype='object')
  warn(f'Новые названия колонок: {df.columns}')


Connection to PostgreSQL DB successful


/Users/affernus/PROJECTS/hacks/postomat_optimisation/src/postamats/utils/connections.py:75: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(f'select * from {table_name}', connection)


,category,name,workinghours,numofseats,numofreaders,numofvisitors,lat,lon,object_type,adm_area,district,address,object_id
0,Библиотека,Библиотека №140,понедельник выходной; вторник 12:00-22:00; сре...,30.0,6103.0,45840.0,55.581234,37.684641,библиотека,Южный административный округ,район Бирюлёво Восточное,"Российская Федерация, город Москва, внутригоро...",6402778e01d99926d24a93a3ccff21168a932ce07a284f...
1,Библиотека,Библиотека №166 имени 1-го Мая,понедельник выходной; вторник 10:00-22:00; сре...,20.0,3002.0,25672.0,55.708619,37.586619,библиотека,Южный административный округ,Донской район,"Российская Федерация, город Москва, внутригоро...",405d8d977e6c63b8442a7fbb7deb2b32d289e02ff5dd9a...
2,Библиотека,Библиотека №169 «Проспект»,понедельник выходной; вторник 12:00-22:00; сре...,34.0,3486.0,30494.0,55.646156,37.477928,библиотека,Юго-Западный административный округ,район Тёплый Стан,"город Москва, Ленинский проспект, дом 127",05b5add41e854523c34bd6baf7f8fd00964d3bee8729b3...
3,Библиотека,Библиотека №266 имени К.И. Чуковского,понедельник выходной; вторник 10:00-19:00; сре...,15.0,678.0,9216.0,55.659028,37.330996,библиотека,Новомосковский административный округ,поселение Внуковское,"город Москва, поселение Внуковское, посёлок ДС...",8cc6a9aea7067457b51bbd4553e119b607f055fa243419...
4,Библиотека,Библиотека №115,понедельник выходной; вторник 12:00-22:00; сре...,52.0,1116.0,22147.0,55.708541,37.782957,библиотека,Юго-Восточный административный округ,район Кузьминки,"Российская Федерация, город Москва, внутригоро...",9ea2fcb8806521ebfcc54d56698d47f0924feb2fa804b1...
...,...,...,...,...,...,...,...,...,...,...,...,...,...
265,Библиотека,Библиотека №222,понедельник выходной; вторник 12:00-22:00; сре...,24.0,6454.0,41860.0,55.654394,37.402099,библиотека,Западный административный округ,район Солнцево,"город Москва, улица Богданова, дом 44",f2cbb972072dec24076aef5fa72e2bbc25ae451305c8f6...
266,Библиотека,Центральная библиотека имени А. С. Пушкина,понедельник выходной; вторник 10:00-22:00; сре...,57.0,827.0,30468.0,NaN,NaN,библиотека,Центральный административный округ,Басманный район,"Российская Федерация, город Москва, внутригоро...",cc56502c03e66a82e2f2f69f2a3db5a5e54d61757bba02...
267,Библиотека,Центральная библиотека №197 имени А. А. Ахматовой,понедельник выходной; вторник 10:00-22:00; сре...,120.0,7325.0,52095.0,55.764945,37.411091,библиотека,Западный административный округ,район Крылатское,"Российская Федерация, город Москва, внутригоро...",e926cd880400e59fa3415a69e3020e4bcb75dd9ba3582d...
268,Библиотека,Дом Лосева,понедельник выходной; вторник 10:00-22:00; сре...,49.0,8500.0,34600.0,55.749545,37.592392,библиотека,Центральный административный округ,район Арбат,"Российская Федерация, город Москва, внутригоро...",83d6ffb04263ebcec89c413e08f2cd509f9db39d1be954...


In [13]:
print(clubs.columns)
clubs = create_lat_lon(clubs, 'geoData')

clubs_cols = ['Category', 'CommonName', 'ObjectAddress', 'WorkingHours', 'LAT', 'LON']
clubs = clubs[clubs_cols].copy()
clubs = explode_address(clubs)

clubs['ObjectType'] = 'дом культуры или клуб'
clubs['WorkingHours'] = clubs['WorkingHours'].apply(extract_working_hours)
clubs[OBJECT_ID_COL] = prepare_data.get_row_hashes(clubs)
clubs.columns = unify_colnames(clubs)
clubs = clubs.drop_duplicates(subset=[OBJECT_ID_COL])
clubs.head(2)

Index(['Category', 'CommonName', 'global_id', 'FullName', 'ShortName',
       'OrgInfo', 'ObjectAddress', 'ChiefOrg', 'ChiefName', 'ChiefPosition',
       'PublicPhone', 'Fax', 'Email', 'WorkingHours',
       'ClarificationOfWorkingHours', 'WebSite', 'geoData'],
      dtype='object')


/Users/affernus/PROJECTS/hacks/postomat_optimisation/src/postamats/utils/prepare_data.py:169: UserWarning: coords is dot
  warn('coords is dot')
/Users/affernus/PROJECTS/hacks/postomat_optimisation/src/postamats/utils/prepare_data.py:172: UserWarning: coords is line
  warn('coords is line')


,CATEGORY,NAME,WORKINGHOURS,LAT,LON,ADM_AREA,DISTRICT,ADDRESS,OBJECT_TYPE,OBJECT_ID
0,Дом культуры или клуб,Культурный центр «Москвич»,понедельник 09:00-21:00; вторник 09:00-21:00; ...,55.707922,37.732911,Юго-Восточный административный округ,район Текстильщики,"Российская Федерация, город Москва, внутригоро...",дом культуры или клуб,3b232147ff5a49326161ddb2b9ee3938d935456b4a2e7c...
1,Дом культуры или клуб,Центр культуры и искусства «Меридиан»,понедельник 10:00-22:00; вторник 10:00-22:00; ...,NaN,NaN,Юго-Западный административный округ,Обручевский район,"Российская Федерация, город Москва, внутригоро...",дом культуры или клуб,e52de5af9f3c45d2d522d38800d01a14fd8fb63c0a5755...


In [14]:
databaase.load_to_bd(clubs, 'clubs_all_data')
databaase.get_table_from_bd('clubs_all_data')

/Users/affernus/PROJECTS/hacks/postomat_optimisation/src/postamats/utils/connections.py:53: UserWarning: В названиях колонок есть uppercase: ['CATEGORY', 'NAME', 'WORKINGHOURS', 'LAT', 'LON', 'ADM_AREA', 'DISTRICT', 'ADDRESS', 'OBJECT_TYPE', 'OBJECT_ID'] postgress имеет проблемы с uppercase, будет выполнен lower()
  warn(
/Users/affernus/PROJECTS/hacks/postomat_optimisation/src/postamats/utils/connections.py:59: UserWarning: Новые названия колонок: Index(['category', 'name', 'workinghours', 'lat', 'lon', 'adm_area',
       'district', 'address', 'object_type', 'object_id'],
      dtype='object')
  warn(f'Новые названия колонок: {df.columns}')


Connection to PostgreSQL DB successful


/Users/affernus/PROJECTS/hacks/postomat_optimisation/src/postamats/utils/connections.py:75: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(f'select * from {table_name}', connection)


,category,name,workinghours,lat,lon,adm_area,district,address,object_type,object_id
0,Дом культуры или клуб,Культурный центр «Москвич»,понедельник 09:00-21:00; вторник 09:00-21:00; ...,55.707922,37.732911,Юго-Восточный административный округ,район Текстильщики,"Российская Федерация, город Москва, внутригоро...",дом культуры или клуб,3b232147ff5a49326161ddb2b9ee3938d935456b4a2e7c...
1,Дом культуры или клуб,Центр культуры и искусства «Меридиан»,понедельник 10:00-22:00; вторник 10:00-22:00; ...,NaN,NaN,Юго-Западный административный округ,Обручевский район,"Российская Федерация, город Москва, внутригоро...",дом культуры или клуб,e52de5af9f3c45d2d522d38800d01a14fd8fb63c0a5755...
2,Дом культуры или клуб,Дом Романса,понедельник 10:00-22:00; вторник 10:00-22:00; ...,55.796019,37.472746,Северо-Западный административный округ,район Щукино,"Российская Федерация, город Москва, внутригоро...",дом культуры или клуб,6e3a4a0292c3dff5ad2ad2920660d523d341d618debab3...
3,Дом культуры или клуб,Государственный музей - культурный центр «Инте...,понедельник 09:00-22:00; вторник 09:00-22:00; ...,55.755105,37.799551,Восточный административный округ,район Ивановское,"Российская Федерация, город Москва, внутригоро...",дом культуры или клуб,402948971a106a37f8121ddfd8d25baadf60a80788ad1f...
4,Дом культуры или клуб,Культурный центр «ЗИЛ»,понедельник 10:00-22:00; вторник 10:00-22:00; ...,55.714461,37.658005,Южный административный округ,Даниловский район,"Российская Федерация, город Москва, внутригоро...",дом культуры или клуб,861f17f1c341a99e7ae2cc3b360b4e5ce1d85c3b2da5e0...
...,...,...,...,...,...,...,...,...,...,...
71,Дом культуры или клуб,Объединение культурных центров Западного админ...,понедельник 09:00-20:00; вторник 09:00-20:00; ...,NaN,NaN,Западный административный округ,район Дорогомилово,"Российская Федерация, город Москва, внутригоро...",дом культуры или клуб,a8dafd284c23223e4c5b9495ec4c042bdff2424609ab91...
72,Дом культуры или клуб,Объединение культурных центров Западного админ...,понедельник 08:00-22:00; вторник 08:00-22:00; ...,NaN,NaN,Западный административный округ,район Кунцево,"Российская Федерация, город Москва, внутригоро...",дом культуры или клуб,1835f86b64bf605e1760c78f30bcc4e08b4ac51b72cb05...
73,Дом культуры или клуб,Объединение культурных центров Западного админ...,понедельник 10:00-21:00; вторник 10:00-21:00; ...,NaN,NaN,Западный административный округ,район Тропарёво-Никулино,"Российская Федерация, город Москва, внутригоро...",дом культуры или клуб,99b522461cd9935726e069db13ec0b64cb9cf2a331c089...
74,Дом культуры или клуб,Объединение культурных центров Западного админ...,понедельник 10:00-22:00; вторник 10:00-22:00; ...,NaN,NaN,Западный административный округ,район Кунцево,"Российская Федерация, город Москва, внутригоро...",дом культуры или клуб,e2093dbe025cb7d348a93824189ca2837e81da22f7b078...


In [15]:
print(sports.columns)
sports = create_lat_lon(sports, 'geoData')
sports_cols = ['Category', 'CommonName', 'ObjectAddress', 'WorkingHours', 'LAT', 'LON']
sports = sports[sports_cols].copy()
sports = explode_address(sports)
sports['WorkingHours'] = sports['WorkingHours'].apply(extract_working_hours)
sports['ObjectType'] = 'cпортивный объект'
sports[OBJECT_ID_COL] = prepare_data.get_row_hashes(sports)
sports.columns = unify_colnames(sports)
sports = sports.drop_duplicates(subset=[OBJECT_ID_COL])
sports.tail(2)

Index(['Category', 'Services', 'ShortName', 'FullName', 'CommonName',
       'OrgInfo', 'ObjectAddress', 'ChiefName', 'ChiefPosition', 'ChiefPhone',
       'PublicPhone', 'Fax', 'Email', 'WebSite', 'WorkingHours',
       'ClarificationWorkingHours', 'global_id', 'geoData', 'geodata_center'],
      dtype='object')


/Users/affernus/PROJECTS/hacks/postomat_optimisation/src/postamats/utils/prepare_data.py:169: UserWarning: coords is dot
  warn('coords is dot')


,CATEGORY,NAME,WORKINGHOURS,LAT,LON,ADM_AREA,DISTRICT,ADDRESS,OBJECT_TYPE,OBJECT_ID
344,Физкультурно-оздоровительный комплекс,"Физкультурно-оздоровительный комплекс (121433,...",понедельник 07:00-23:00; вторник 07:00-23:00; ...,55.734761,37.456232,Западный административный округ,район Фили-Давыдково,"Российская Федерация, город Москва, внутригоро...",cпортивный объект,57b14501dcf080327f2007b1d576220cbd40fba0028f6e...
345,Стадион (1500 и более мест),Стадион «Москвич»,понедельник 07:00-23:00; вторник 07:00-23:00; ...,55.708107,37.735709,Юго-Восточный административный округ,район Текстильщики,"г. Москва, пр-кт. Волгоградский, дом 46/15, ст...",cпортивный объект,645902454fd6a8e508c0ea7f275860e197cf1714e9d36f...


In [16]:
databaase.load_to_bd(sports, 'sports_all_data')
databaase.get_table_from_bd('sports_all_data')

/Users/affernus/PROJECTS/hacks/postomat_optimisation/src/postamats/utils/connections.py:53: UserWarning: В названиях колонок есть uppercase: ['CATEGORY', 'NAME', 'WORKINGHOURS', 'LAT', 'LON', 'ADM_AREA', 'DISTRICT', 'ADDRESS', 'OBJECT_TYPE', 'OBJECT_ID'] postgress имеет проблемы с uppercase, будет выполнен lower()
  warn(
/Users/affernus/PROJECTS/hacks/postomat_optimisation/src/postamats/utils/connections.py:59: UserWarning: Новые названия колонок: Index(['category', 'name', 'workinghours', 'lat', 'lon', 'adm_area',
       'district', 'address', 'object_type', 'object_id'],
      dtype='object')
  warn(f'Новые названия колонок: {df.columns}')


Connection to PostgreSQL DB successful


/Users/affernus/PROJECTS/hacks/postomat_optimisation/src/postamats/utils/connections.py:75: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(f'select * from {table_name}', connection)


,category,name,workinghours,lat,lon,adm_area,district,address,object_type,object_id
0,Физкультурно-оздоровительный комплекс,Физкультурно-оздоровительный комплекс «ГБУ «СШ...,None,55.639497,37.359522,None,None,None,cпортивный объект,ce537128134b2e518211d2d3dd3864dedf8537d9b5f22d...
1,Крытое сооружение с искусственным льдом,Крытое сооружение с искусственным льдом «Радуж...,None,55.645979,37.351461,None,None,None,cпортивный объект,4f9ce922d13ad916c3ee40c29632f6004979fd40184ddc...
2,Физкультурно-оздоровительный комплекс,Физкультурно-оздоровительный комплекс «ГБУ «СШ...,None,55.809485,37.495732,None,None,None,cпортивный объект,11901b95a1041863779c6d4c8f0da11a005c2ee0bb6eec...
3,Физкультурно-оздоровительный комплекс,Спортивный комплекс «На Таганке» (105927),понедельник 08:00-22:00; вторник 08:00-22:00; ...,55.742406,37.684085,Центральный административный округ,Таганский район,"Российская Федерация, город Москва, внутригоро...",cпортивный объект,5747ffc6f32438d1d227a62b21977c6d20cbdbc9729178...
4,Спортивный зал,Спортивный зал «ГБУ «СШ № 76» Москомспорта»,None,55.862233,37.562579,None,None,None,cпортивный объект,4f3aa6164b8db76cea81b37ba6b4daedd16e6b2cff2f51...
...,...,...,...,...,...,...,...,...,...,...
341,Спортивный зал,Спортивный зал «Зеленоград 1613»,понедельник 09:00-21:00; вторник 09:00-21:00; ...,55.974472,37.155215,Зеленоградский административный округ,район Крюково,"Российская Федерация, город Москва, внутригоро...",cпортивный объект,632a2336a5d1ff329573ddc4b2eaf9be67f72929d88b5d...
342,Велодром,ВМХ-велодром «Зеленоград»,понедельник 09:00-21:00; вторник 09:00-21:00; ...,55.962885,37.174221,Зеленоградский административный округ,район Крюково,"город Москва, город Зеленоград, проектируемый ...",cпортивный объект,b37ae820b551f2becd5877619bdba042c62a5de30c1a37...
343,Спортивный зал,Спортивный зал «Орехово»,понедельник 09:00-21:00; вторник 09:00-22:00; ...,55.614951,37.713617,Южный административный округ,район Орехово-Борисово Северное,"Российская Федерация, город Москва, внутригоро...",cпортивный объект,ded1a72a2eb576a1f96070e6e8d7b3cd72da693414e087...
344,Физкультурно-оздоровительный комплекс,"Физкультурно-оздоровительный комплекс (121433,...",понедельник 07:00-23:00; вторник 07:00-23:00; ...,55.734761,37.456232,Западный административный округ,район Фили-Давыдково,"Российская Федерация, город Москва, внутригоро...",cпортивный объект,57b14501dcf080327f2007b1d576220cbd40fba0028f6e...


In [17]:
apartment_houses = databaase.get_table_from_bd('apartment_houses_all_data')

Connection to PostgreSQL DB successful


/Users/affernus/PROJECTS/hacks/postomat_optimisation/src/postamats/utils/connections.py:75: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(f'select * from {table_name}', connection)


In [26]:
# костыль - изначальный нейминг был в uppercase, но postgress имеет с ним проблемы
# потому при загрузке все колонки переводятся в lower()
# переводим всё обратно, чтобы соответствовать формату тетрадки
apartment_houses.columns = [col.upper() for col in apartment_houses.columns]

In [27]:
all_objects_data = pd.concat([apartment_houses, kiosks, mfc, libs, clubs, sports], ignore_index=True)
all_objects_data = all_objects_data[MANDATORY_COLS]

In [28]:
all_objects_data

,OBJECT_ID,ADM_AREA,DISTRICT,OBJECT_TYPE,LAT,LON,ADDRESS
0,a1b50bfc376ea37edebd16ddd131b106bf8d8c846983a9...,Центральный административный округ,муниципальный округ Замоскворечье,многоквартирный дом,55.735592,37.634462,"Российская Федерация, город Москва, внутригоро..."
1,9a11acf2996873fc758e8f2454f47250bd5239153c8dc8...,Западный административный округ,муниципальный округ Фили-Давыдково,многоквартирный дом,55.720403,37.457119,"Российская Федерация, город Москва, внутригоро..."
2,4c4a73fa11c8817c1726cd5e0762088ddfa0db1e92032f...,Северо-Западный административный округ,муниципальный округ Куркино,многоквартирный дом,55.906276,37.378763,"город Москва, Соколово-Мещерская улица, дом 29..."
3,1899f036c5adf9e920a84e42d0e2b95d3d45f52a9a1899...,Северо-Западный административный округ,муниципальный округ Куркино,многоквартирный дом,55.906924,37.380579,"Российская Федерация, город Москва, внутригоро..."
4,ecd4be6fbdf3567b224e3aa38d8f639dd0378a3e0d9021...,Южный административный округ,муниципальный округ Донской,многоквартирный дом,55.706783,37.602183,"Российская Федерация, город Москва, внутригоро..."
...,...,...,...,...,...,...,...
43178,632a2336a5d1ff329573ddc4b2eaf9be67f72929d88b5d...,Зеленоградский административный округ,район Крюково,cпортивный объект,55.974472,37.155215,"Российская Федерация, город Москва, внутригоро..."
43179,b37ae820b551f2becd5877619bdba042c62a5de30c1a37...,Зеленоградский административный округ,район Крюково,cпортивный объект,55.962885,37.174221,"город Москва, город Зеленоград, проектируемый ..."
43180,ded1a72a2eb576a1f96070e6e8d7b3cd72da693414e087...,Южный административный округ,район Орехово-Борисово Северное,cпортивный объект,55.614951,37.713617,"Российская Федерация, город Москва, внутригоро..."
43181,57b14501dcf080327f2007b1d576220cbd40fba0028f6e...,Западный административный округ,район Фили-Давыдково,cпортивный объект,55.734761,37.456232,"Российская Федерация, город Москва, внутригоро..."


In [29]:
assert all_objects_data[OBJECT_ID_COL].duplicated().sum() == 0

In [30]:
all_objects_data

,OBJECT_ID,ADM_AREA,DISTRICT,OBJECT_TYPE,LAT,LON,ADDRESS
0,a1b50bfc376ea37edebd16ddd131b106bf8d8c846983a9...,Центральный административный округ,муниципальный округ Замоскворечье,многоквартирный дом,55.735592,37.634462,"Российская Федерация, город Москва, внутригоро..."
1,9a11acf2996873fc758e8f2454f47250bd5239153c8dc8...,Западный административный округ,муниципальный округ Фили-Давыдково,многоквартирный дом,55.720403,37.457119,"Российская Федерация, город Москва, внутригоро..."
2,4c4a73fa11c8817c1726cd5e0762088ddfa0db1e92032f...,Северо-Западный административный округ,муниципальный округ Куркино,многоквартирный дом,55.906276,37.378763,"город Москва, Соколово-Мещерская улица, дом 29..."
3,1899f036c5adf9e920a84e42d0e2b95d3d45f52a9a1899...,Северо-Западный административный округ,муниципальный округ Куркино,многоквартирный дом,55.906924,37.380579,"Российская Федерация, город Москва, внутригоро..."
4,ecd4be6fbdf3567b224e3aa38d8f639dd0378a3e0d9021...,Южный административный округ,муниципальный округ Донской,многоквартирный дом,55.706783,37.602183,"Российская Федерация, город Москва, внутригоро..."
...,...,...,...,...,...,...,...
43178,632a2336a5d1ff329573ddc4b2eaf9be67f72929d88b5d...,Зеленоградский административный округ,район Крюково,cпортивный объект,55.974472,37.155215,"Российская Федерация, город Москва, внутригоро..."
43179,b37ae820b551f2becd5877619bdba042c62a5de30c1a37...,Зеленоградский административный округ,район Крюково,cпортивный объект,55.962885,37.174221,"город Москва, город Зеленоград, проектируемый ..."
43180,ded1a72a2eb576a1f96070e6e8d7b3cd72da693414e087...,Южный административный округ,район Орехово-Борисово Северное,cпортивный объект,55.614951,37.713617,"Российская Федерация, город Москва, внутригоро..."
43181,57b14501dcf080327f2007b1d576220cbd40fba0028f6e...,Западный административный округ,район Фили-Давыдково,cпортивный объект,55.734761,37.456232,"Российская Федерация, город Москва, внутригоро..."


In [31]:
all_objects_data['OBJECT_TYPE'].value_counts()

многоквартирный дом      38023
киоск                     4329
cпортивный объект          346
библиотека                 270
МФЦ                        139
дом культуры или клуб       76
Name: OBJECT_TYPE, dtype: int64

In [32]:
assert all_objects_data['OBJECT_TYPE'].unique().shape[0] == 6

In [33]:
databaase.load_to_bd(all_objects_data, 'all_objects_data')
databaase.get_table_from_bd('all_objects_data')

/Users/affernus/PROJECTS/hacks/postomat_optimisation/src/postamats/utils/connections.py:53: UserWarning: В названиях колонок есть uppercase: ['OBJECT_ID', 'ADM_AREA', 'DISTRICT', 'OBJECT_TYPE', 'LAT', 'LON', 'ADDRESS'] postgress имеет проблемы с uppercase, будет выполнен lower()
  warn(
/Users/affernus/PROJECTS/hacks/postomat_optimisation/src/postamats/utils/connections.py:59: UserWarning: Новые названия колонок: Index(['object_id', 'adm_area', 'district', 'object_type', 'lat', 'lon',
       'address'],
      dtype='object')
  warn(f'Новые названия колонок: {df.columns}')


Connection to PostgreSQL DB successful


/Users/affernus/PROJECTS/hacks/postomat_optimisation/src/postamats/utils/connections.py:75: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(f'select * from {table_name}', connection)


,object_id,adm_area,district,object_type,lat,lon,address
0,a1b50bfc376ea37edebd16ddd131b106bf8d8c846983a9...,Центральный административный округ,муниципальный округ Замоскворечье,многоквартирный дом,55.735592,37.634462,"Российская Федерация, город Москва, внутригоро..."
1,9a11acf2996873fc758e8f2454f47250bd5239153c8dc8...,Западный административный округ,муниципальный округ Фили-Давыдково,многоквартирный дом,55.720403,37.457119,"Российская Федерация, город Москва, внутригоро..."
2,4c4a73fa11c8817c1726cd5e0762088ddfa0db1e92032f...,Северо-Западный административный округ,муниципальный округ Куркино,многоквартирный дом,55.906276,37.378763,"город Москва, Соколово-Мещерская улица, дом 29..."
3,1899f036c5adf9e920a84e42d0e2b95d3d45f52a9a1899...,Северо-Западный административный округ,муниципальный округ Куркино,многоквартирный дом,55.906924,37.380579,"Российская Федерация, город Москва, внутригоро..."
4,ecd4be6fbdf3567b224e3aa38d8f639dd0378a3e0d9021...,Южный административный округ,муниципальный округ Донской,многоквартирный дом,55.706783,37.602183,"Российская Федерация, город Москва, внутригоро..."
...,...,...,...,...,...,...,...
43178,632a2336a5d1ff329573ddc4b2eaf9be67f72929d88b5d...,Зеленоградский административный округ,район Крюково,cпортивный объект,55.974472,37.155215,"Российская Федерация, город Москва, внутригоро..."
43179,b37ae820b551f2becd5877619bdba042c62a5de30c1a37...,Зеленоградский административный округ,район Крюково,cпортивный объект,55.962885,37.174221,"город Москва, город Зеленоград, проектируемый ..."
43180,ded1a72a2eb576a1f96070e6e8d7b3cd72da693414e087...,Южный административный округ,район Орехово-Борисово Северное,cпортивный объект,55.614951,37.713617,"Российская Федерация, город Москва, внутригоро..."
43181,57b14501dcf080327f2007b1d576220cbd40fba0028f6e...,Западный административный округ,район Фили-Давыдково,cпортивный объект,55.734761,37.456232,"Российская Федерация, город Москва, внутригоро..."
